In [1]:
# Read eidCodesMap from json

import json

work_loc = '/run/media/anton/Elements/UKB'

# Get number of codes

with open(f'{work_loc}/codes.json', 'r') as f:
    codes = json.load(f)
    D = len(codes)
    print(D)

with open(f'{work_loc}/eidCodesMap.json', 'r') as f:
    eidCodesMap = json.load(f)

print(len(eidCodesMap))

for k in eidCodesMap:
    print(eidCodesMap[k])
    break

1558
33204
[[870, 870, 881], [748]]


In [2]:
print(k)

1000217


In [11]:
# Read pca fcs

import pickle

pca_loc = '/run/media/anton/Elements/UKB/fc_pca/fcs_pca200.pkl'

fc_pca = pickle.load(open(pca_loc, 'rb'))
print(len(fc_pca))

38679


In [13]:
for k in fc_pca:
    print(k)
    break

for k in eidCodesMap:
    print(k)
    break

1000217
1000217


In [15]:
# Prediction from FC PCA

xpca = []
ypca = []

for k,v in eidCodesMap.items():
    xone = fc_pca[int(k)]
    yone = np.zeros(D)
    if len(v) < 1:
        continue
    if v[-1] is None:
        continue
    for code in v[-1]:
        yone[code] = 1
    xpca.append(xone)
    ypca.append(yone)

xpca = np.stack(xpca)
ypca = np.stack(ypca)

print(xpca.shape)
print(ypca.shape)

(33204, 200)
(33204, 1558)


In [2]:
import numpy as np

x = []
y = []

for k,v in eidCodesMap.items():
    if len(v) < 2:
        continue
    xone = np.zeros(D)
    yone = np.zeros(D)
    atleast_one = False
    for i in range(len(v)-1):
        if v[i] is None:
            # print(f'Problem with {k}')
            continue
        for code in v[i]:
            xone[code] += 1
            atleast_one = True
    if not atleast_one:
        continue
    if v[-1] is None:
        # print(f'Another problem with {k}')
        continue
    for code in v[-1]:
        yone[code] = 1
    x.append(xone)
    y.append(yone)

x = np.stack(x)
y = np.stack(y)

print(x.shape)
print(y.shape)

(27461, 1558)
(27461, 1558)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

xtr, xt, ytr, yt = train_test_split(x, y, train_size=10000)

for i in range(ytr.shape[1]):
    ytri = ytr[:,i]
    yti = yt[:,i]
    if not ytri.any() or not yti.any():
        continue
    reg = LogisticRegression(C=100, max_iter=1000).fit(xtr, ytri)
    yhat = reg.predict(xt)
    tn, fp, fn, tp = confusion_matrix(yti, yhat).ravel()
    sp = tn / (tn+fp)
    sn = tp / (tp+fn)
    print(i, codes[i], sp, sn)

1 A02 1.0 0.0
3 A04 0.9979343585035575 0.0
7 A08 0.999541626081476 0.0
8 A09 0.9880883543425465 0.005988023952095809
16 A40 0.9995990376904571 0.0
17 A41 0.9950486498934884 0.05434782608695652
21 A49 0.9987963546741561 0.07142857142857142
37 B00 0.9998281491665234 0.0
39 B02 0.9991974777873316 0.1875
41 B07 0.9979366080128389 0.0
47 B18 0.99982809993124 0.2222222222222222
56 B34 0.998795388057133 0.03571428571428571
57 B35 0.9999427032601845 0.0
59 B37 0.9976461132162131 0.023255813953488372
60 B44 0.9998281393217232 0.0
68 B59 1.0 0.0
71 B80 1.0 0.0
79 B94 0.9995417573605224 0.0
80 B95 0.9984509466437177 0.0967741935483871
81 B96 0.9949372914509262 0.0759493670886076
82 B97 0.9958058029301925 0.32142857142857145
83 B98 0.9981644008489646 0.0
84 B99 0.9997135491263248 0.0
86 C01 0.9997135655362053 0.0
87 C02 1.0 0.0
91 C07 0.9997708918036543 0.0
93 C09 0.9991980294437761 0.5
97 C15 0.9994268355591219 0.7857142857142857
98 C16 0.99959889983956 0.1111111111111111
99 C17 0.999885445901827

In [9]:
# Predict last codes if any of previous time points had that code

for i in range(ytr.shape[1]):
    yti = yt[:,i]
    if not yti.any():
        continue
    yhat = np.zeros(yti.shape)
    yhat[xt[:,i].nonzero()] = 1
    tn, fp, fn, tp = confusion_matrix(yti, yhat).ravel()
    sp = tn / (tn+fp)
    sn = tp / (tp+fn)
    print(i, codes[i], sp, sn)

1 A02 0.9997708786802612 0.0
2 A03 0.999942726231386 1.0
3 A04 0.9940899701629562 0.2727272727272727
4 A05 0.999942726231386 0.0
6 A07 0.9998281786941581 0.0
7 A08 0.9979941543928019 0.5
8 A09 0.97084345713294 0.18857142857142858
16 A40 0.9994844474995703 0.75
17 A41 0.9878624022089277 0.6493506493506493
19 A46 0.9994845360824742 1.0
20 A48 0.9998281688527407 0.0
21 A49 0.9974777873316136 0.5625
25 A63 0.9990836197021764 0.0
26 A69 0.9997135983503265 0.3333333333333333
32 A85 0.999942726231386 1.0
37 B00 0.9993125966660938 0.0
39 B02 0.9982229865290915 0.625
41 B07 0.9969043797294198 0.0
42 B08 0.9997136311569301 0.0
43 B09 0.999942726231386 0.0
45 B16 0.9998281786941581 0.0
46 B17 0.9993699885452463 0.0
47 B18 0.9991405981436919 0.7142857142857143
52 B24 0.9997708918036543 0.5
56 B34 0.9928878692285632 0.15384615384615385
57 B35 0.9988541965052994 0.16666666666666666
59 B37 0.9930547583515096 0.1794871794871795
60 B44 0.9996562392574768 0.7142857142857143
68 B59 0.9997708918036543 0.5

In [17]:
# Predict using just FC pca

xtr_pca, xt_pca, ytr_pca, yt_pca = train_test_split(xpca, ypca, train_size=10000)

for i in range(ytr_pca.shape[1]):
    ytri = ytr_pca[:,i]
    yti = yt_pca[:,i]
    if not ytri.any() or not yti.any():
        continue
    reg = LogisticRegression(C=1000, max_iter=1000).fit(xtr_pca, ytri)
    yhat = reg.predict(xt_pca)
    tn, fp, fn, tp = confusion_matrix(yti, yhat).ravel()
    sp = tn / (tn+fp)
    sn = tp / (tp+fn)
    print(i, codes[i], sp, sn)

1 A02 1.0 0.0
3 A04 0.9977977372830124 0.021739130434782608
6 A07 0.9999569021247252 0.0
7 A08 0.9997844084166954 0.0
8 A09 0.9997388920318552 0.0
16 A40 0.9998706896551725 0.0
17 A41 0.9946341252325933 0.031578947368421054
20 A48 1.0 0.0
21 A49 0.9995687796463993 0.0
26 A69 1.0 0.0
37 B00 1.0 0.0
39 B02 0.9996549939623943 0.0
41 B07 0.9991806106606865 0.0
47 B18 0.9996550980814831 0.0
56 B34 0.9895995166580356 0.0
57 B35 1.0 0.0
59 B37 0.9958110209017101 0.0
60 B44 0.9999568946937368 0.0
71 B80 1.0 0.0
79 B94 1.0 0.0
80 B95 0.9953379953379954 0.0
81 B96 0.9983991000346141 0.0
82 B97 0.9875075646234979 0.04285714285714286
83 B98 0.9927070297328788 0.0
84 B99 0.9999137931034483 0.0
85 C00 1.0 0.0
87 C02 1.0 0.0
88 C03 0.9999569021247252 0.0
92 C08 1.0 0.0
93 C09 0.9999568946937368 0.0
97 C15 0.9997843340234644 0.0
98 C16 0.9999568909772816 0.0
100 C18 0.996889042516419 0.0
101 C19 1.0 0.0
102 C20 0.9989218096347091 0.0
103 C21 0.9999137968191026 0.0
104 C22 0.999654964202536 0.0
105 C23